# import python packages

In [149]:
import pandas as pd
import glob, os
from pyswmm import Simulation, Links
# import multiprocessing as mp
import matplotlib.pyplot as plt
import fastparquet as fp

# read SWMM filenames

In [140]:
swmm_files = os.listdir('../Output_SWMM_directory/')
swmm_files = pd.DataFrame(swmm_files, columns=['swmm_filename'])
swmm_files.sort_values(by='swmm_filename', inplace=True, ignore_index=True)
swmm_files = swmm_files.iloc[:100,:]
swmm_files

,swmm_filename
0,frequent______freq_0_2_EY___________010__01.inp
1,frequent______freq_0_2_EY___________010__02.inp
2,frequent______freq_0_2_EY___________010__03.inp
3,frequent______freq_0_2_EY___________010__04.inp
4,frequent______freq_0_2_EY___________010__05.inp
...,...
95,frequent______freq_0_5_EY___________010__06.inp
96,frequent______freq_0_5_EY___________010__07.inp
97,frequent______freq_0_5_EY___________010__08.inp
98,frequent______freq_0_5_EY___________010__09.inp


# delete files to avoid errors

In [141]:
for f in glob.glob('../Output_SWMM_directory/*.parquet'):
    os.remove(f)

for f in glob.glob('../Output_SWMM_directory/*.out'):
    os.remove(f)

for f in glob.glob('../Output_SWMM_directory/*.rpt'):
    os.remove(f)

# simulate SWMM files, export SWMM results

In [142]:
hydrograph_list = []

for index_1 in swmm_files.index:
    a1 = swmm_files.swmm_filename[index_1]
    with Simulation(r'../Output_SWMM_directory/' + a1) as sim:
        Link116 = Links(sim)['116']
        temp_list = []
        for index_2, step in enumerate(sim):
            a2 = pd.DataFrame({'timestamp':[sim.current_time], 'flow':[Link116.flow]})
            temp_list.append(a2)
        temp_list = pd.concat(temp_list, ignore_index=True)
        temp_list.to_parquet('../Output_SWMM_directory/' + a1[:-4] + '.parquet')
        a3 = pd.DataFrame({'frequency':[a1[:-4][:14].split('__')[0]], 
                           'freq_AEP':[a1[:-4][14:-7].split('__')[0]], 
                           'duration':[int(a1[:-4][-7:-4])], 
                           'pattern':[int(a1[:-4][-2:])], 
                           'hydrograph':[temp_list]})
        hydrograph_list.append(a3)

hydrograph_list = pd.concat(hydrograph_list, ignore_index=True)
hydrograph_list

,frequency,freq_AEP,duration,pattern,hydrograph
0,frequent,freq_0_2_EY,10,1,timestamp flow 0 2023-01...
1,frequent,freq_0_2_EY,10,2,timestamp flow 0 2023-01...
2,frequent,freq_0_2_EY,10,3,timestamp flow 0 2023-01...
3,frequent,freq_0_2_EY,10,4,timestamp flow 0 2023-01...
4,frequent,freq_0_2_EY,10,5,timestamp flow 0 2023-01...
...,...,...,...,...,...
95,frequent,freq_0_5_EY,10,6,timestamp flow 0 2023-01...
96,frequent,freq_0_5_EY,10,7,timestamp flow 0 2023-01...
97,frequent,freq_0_5_EY,10,8,timestamp flow 0 2023-01...
98,frequent,freq_0_5_EY,10,9,timestamp flow 0 2023-01...


# get peak flow

In [143]:
max_flow_list = []

for index_3 in hydrograph_list.index:
    b1 = hydrograph_list.hydrograph[index_3].flow.max()
    max_flow_list.append(b1)

max_flow_list = pd.DataFrame({'max_flow':max_flow_list})
max_flow_list = pd.concat([hydrograph_list.iloc[:,:4], max_flow_list], axis=1)
max_flow_list

,frequency,freq_AEP,duration,pattern,max_flow
0,frequent,freq_0_2_EY,10,1,14.064574
1,frequent,freq_0_2_EY,10,2,14.360415
2,frequent,freq_0_2_EY,10,3,13.859452
3,frequent,freq_0_2_EY,10,4,14.387873
4,frequent,freq_0_2_EY,10,5,14.211152
...,...,...,...,...,...
95,frequent,freq_0_5_EY,10,6,12.625750
96,frequent,freq_0_5_EY,10,7,12.415615
97,frequent,freq_0_5_EY,10,8,12.225841
98,frequent,freq_0_5_EY,10,9,12.928225


In [148]:
max_flow_list.groupby(by=max_flow_list.columns[1:3].to_list()).agg({'max_flow':['describe']})

max_flow                                             \
                     describe                                              
                        count       mean       std        min        25%   
freq_AEP    duration                                                       
freq_0_2_EY 10           10.0  14.409101  0.319253  13.859452  14.248467   
            15           10.0  15.853758  0.482017  14.875977  15.868283   
            20           10.0  15.792678  0.911690  14.507990  15.052340   
            25           10.0  15.151653  1.031341  13.425577  14.643342   
            30           10.0  14.986001  1.130143  12.908811  14.414444   
            45           10.0  15.041120  1.083096  13.483744  14.361492   
            60           10.0  12.719237  2.257819   9.430740  11.355384   
            90           10.0  10.638932  2.464843   7.476431   8.598229   
            120          10.0  10.794070  2.751545   7.782621   8.513497   
freq_0_5_EY 10           10.0  12.250250  0.354563  11.750100  12.040095   

                                                       
                                                       
                            50%        75%        max  
freq_AEP    duration                                   
freq_0_2_EY 10        14.414546  14.522274  14.941526  
            15        15.980652  16.076996  16.437045  
            20        15.923456  16.414486  17.294208  
            25        15.439248  16.027961  16.137153  
            30        14.785836  15.926431  16.633668  
            45        14.965373  15.857645  16.549569  
            60        12.360285  14.870427  15.771910  
            90        10.557450  12.398723  14.693831  
            120       10.358530  12.567664  15.808351  
freq_0_5_EY 10        12.185913  12.413018  12.928225

In [ ]:
plt.boxplot()